In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

# Ecopetrol

## DATOS

In [ ]:
indice_referencia = "ECOPETROL.CL"
df_indice = pd.read_csv(f"Resultados_Experimento/{indice_referencia}/stock_sugerencias.csv")
df_indice=df_indice.loc[: ,["Date","Buy_Mantein_Rnd","Revertion_Mean_Rnd","Media_Movil_Rnd","Agente_BVC_Rnd"]]
df_indice.head(5)

,Date,Buy_Mantein_Rnd,Revertion_Mean_Rnd,Media_Movil_Rnd,Agente_BVC_Rnd
0,2024-01-01 00:00:00-05:00,0.000000,0.0,0.000000,0.000000
1,2024-01-02 00:00:00-05:00,0.006410,0.0,-0.006410,-0.006410
2,2024-01-03 00:00:00-05:00,0.027601,0.0,-0.027601,0.027601
3,2024-01-04 00:00:00-05:00,-0.006198,0.0,0.006198,0.000000
4,2024-01-05 00:00:00-05:00,0.016632,0.0,-0.016632,0.000000


## Metricas y ANOVA

In [ ]:

# df_indice ya está cargado con las columnas mencionadas
df_indice['Date'] = pd.to_datetime(df_indice['Date'])

# Transformar el DataFrame a un formato largo para análisis
df_melted = df_indice.melt(id_vars=['Date'], 
                            var_name='Estrategia', 
                            value_name='Rendimiento')

# Calcular métricas por estrategia
df_metrica = df_melted.groupby('Estrategia').agg(
    ARR=('Rendimiento', lambda x: ((1+np.mean(x)) ** (253)) - 1),
    Volatilidad=('Rendimiento', lambda x: np.std(x) * np.sqrt(253)),
    Sharpe=('Rendimiento', lambda x: ((((1+np.mean(x)) ** (253)) - 1)-0.05) / (np.std(x) * np.sqrt(253)))
).reset_index()

print("Métricas calculadas por estrategia:")
print(df_metrica)

# ANOVA para cada métrica
print("\nResultados ANOVA:")
for metric in ['ARR', 'Volatilidad', 'Sharpe']:
    groups = [df_melted[df_melted['Estrategia'] == strat]['Rendimiento'] for strat in df_melted['Estrategia'].unique()]
    f_stat, p_value = stats.f_oneway(*groups)
    print(f"{metric}: F={f_stat:.3f}, p-value={p_value:.3f}")


Métricas calculadas por estrategia:
           Estrategia       ARR  Volatilidad    Sharpe
0      Agente_BVC_Rnd -0.147437     0.185098 -1.066664
1     Buy_Mantein_Rnd -0.197879     0.278315 -0.890643
2     Media_Movil_Rnd  0.034492     0.278652 -0.055653
3  Revertion_Mean_Rnd -0.424042     0.138962 -3.411304

Resultados ANOVA:
ARR: F=1.131, p-value=0.335
Volatilidad: F=1.131, p-value=0.335
Sharpe: F=1.131, p-value=0.335


C:\Users\jumar\AppData\Local\Temp\ipykernel_952\3783521943.py:2: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



## Evolución Metricas

In [ ]:
# Transformar el DataFrame a formato largo
df_melted = df_indice.melt(id_vars=['Date'], var_name='Estrategia', value_name='Rendimiento')

# Función para calcular métricas acumulativas
def calcular_metricas_acumulativas(df):
    df = df.sort_values('Date').copy()  # Asegurar orden cronológico
    df['ARR_acum'] = (1 + df['Rendimiento']).cumprod() ** (253 / df.index.to_series().diff().fillna(1).cumsum()) - 1
    df['Volatilidad_acum'] = df['Rendimiento'].expanding().std() * np.sqrt(252)
    df['Sharpe_acum'] = (df['Rendimiento'].expanding().mean() / df['Rendimiento'].expanding().std()) * np.sqrt(252)
    return df

# Aplicar función por estrategia
df_time_metrics = df_melted.groupby('Estrategia', group_keys=False).apply(calcular_metricas_acumulativas)

fig = go.Figure()

for metric in ['ARR_acum']:
    for strategy in df_time_metrics['Estrategia'].unique():
        df_temp = df_time_metrics[df_time_metrics['Estrategia'] == strategy]
        fig.add_trace(go.Scatter(
            x=df_temp['Date'],
            y=df_temp[metric],
            mode='lines',
            name=f"{strategy} - {metric}"
        ))

fig.update_layout(
    title={
        'text': f"<b>Evolución Acumulativa del ARR para {indice_referencia}</b>",
        'x':0.5,
        'xanchor': 'center'
    },
    yaxis_title="<b>ARR Acumulado</b>",
    legend=dict(
        title="<b>Estrategia</b>",
        orientation="h",
        yanchor="top",
        y=-0.15,
        xanchor="center",
        x=0.5
    ),
    template="ggplot2"
)


fig.show()


C:\Users\jumar\AppData\Local\Temp\ipykernel_952\4236578591.py:13: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



## Estadisticas

In [38]:
from scipy.stats import shapiro

print("Prueba de Normalidad (Shapiro-Wilk):")
for estrategia in df_melted['Estrategia'].unique():
    data = df_melted[df_melted['Estrategia'] == estrategia]['Rendimiento']
    stat, p_value = shapiro(data)
    normalidad = "Normal" if p_value > 0.05 else "No normal"
    print(f"{estrategia}: Estadístico={stat:.3f}, p-value={p_value:.3f} ({normalidad})")


Prueba de Normalidad (Shapiro-Wilk):
Buy_Mantein_Rnd: Estadístico=0.967, p-value=0.000 (No normal)
Revertion_Mean_Rnd: Estadístico=0.277, p-value=0.000 (No normal)
Media_Movil_Rnd: Estadístico=0.968, p-value=0.000 (No normal)
Agente_BVC_Rnd: Estadístico=0.685, p-value=0.000 (No normal)


In [39]:
from scipy.stats import levene

# Preparar grupos para Levene
grupos = [df_melted[df_melted['Estrategia'] == estr]['Rendimiento'] 
          for estr in df_melted['Estrategia'].unique()]

# Ejecutar prueba de Levene
estadistico, p_value = levene(*grupos)

homocedasticidad = "Homocedástico" if p_value > 0.05 else "Heterocedástico"
print(f"\nPrueba de Homocedasticidad (Levene): Estadístico={estadistico:.3f}, p-value={p_value:.3f} ({homocedasticidad})")



Prueba de Homocedasticidad (Levene): Estadístico=55.826, p-value=0.000 (Heterocedástico)


In [ ]:
from scipy.stats import mannwhitneyu
from itertools import combinations
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# 'df_melted' con las columnas 'Estrategia' y 'Rendimiento'

# Extrae las estrategias únicas
estrategias = df_melted['Estrategia'].unique()

# Número de comparaciones (para la corrección de Bonferroni)
n_comparaciones = len(list(combinations(estrategias, 2)))

# Nivel de significancia ajustado (corrección de Bonferroni)
alpha_ajustado = 0.05 / n_comparaciones

# Realiza las comparaciones por pares con Mann-Whitney U
print("Resultados prueba Mann-Whitney U por pares (con corrección de Bonferroni):")
for estrategia1, estrategia2 in combinations(estrategias, 2):
    # Filtra los rendimientos por cada estrategia
    rend_1 = df_melted[df_melted['Estrategia'] == estrategia1]['Rendimiento']
    rend_2 = df_melted[df_melted['Estrategia'] == estrategia2]['Rendimiento']

    # Aplica Mann-Whitney U test
    stat, p_value = mannwhitneyu(rend_1, rend_2, alternative='two-sided')

    # Verifica la significancia estadística con la corrección de Bonferroni
    if p_value <= alpha_ajustado:
        significativo = "Sí"
    else:
        significativo = "No"

    print(f"{estrategia1} vs {estrategia2}: Estadístico U={stat:.3f}, p-valor={p_value:.3f}, Significativo={significativo}")

Resultados prueba Mann-Whitney U por pares (con corrección de Bonferroni):
Buy_Mantein_Rnd vs Revertion_Mean_Rnd: Estadístico U=34086.500, p-valor=0.173, Significativo=No
Buy_Mantein_Rnd vs Media_Movil_Rnd: Estadístico U=31276.500, p-valor=0.658, Significativo=No
Buy_Mantein_Rnd vs Agente_BVC_Rnd: Estadístico U=32200.500, p-valor=0.903, Significativo=No
Revertion_Mean_Rnd vs Media_Movil_Rnd: Estadístico U=30556.000, p-valor=0.343, Significativo=No
Revertion_Mean_Rnd vs Agente_BVC_Rnd: Estadístico U=29266.500, p-valor=0.021, Significativo=No
Media_Movil_Rnd vs Agente_BVC_Rnd: Estadístico U=32052.000, p-valor=0.977, Significativo=No


# NUTRESA

## DATOS

In [ ]:
indice_referencia = "NUTRESA.CL"
df_indice = pd.read_csv(f"Resultados_Experimento/{indice_referencia}/stock_sugerencias.csv")
df_indice=df_indice.loc[: ,["Date","Buy_Mantein_Rnd","Revertion_Mean_Rnd","Media_Movil_Rnd","Agente_BVC_Rnd"]]
df_indice.head(5)

,Date,Buy_Mantein_Rnd,Revertion_Mean_Rnd,Media_Movil_Rnd,Agente_BVC_Rnd
0,2024-01-01 00:00:00-05:00,0.000000,0.0,0.000000,0.000000
1,2024-01-02 00:00:00-05:00,0.017333,0.0,-0.017333,0.000000
2,2024-01-03 00:00:00-05:00,0.015727,0.0,-0.015727,0.000000
3,2024-01-04 00:00:00-05:00,-0.012903,0.0,0.012903,-0.012903
4,2024-01-05 00:00:00-05:00,-0.020044,0.0,0.020044,-0.020044


## Metricas y ANOVA

In [ ]:
# df_indice ya está cargado con las columnas mencionadas
df_indice['Date'] = pd.to_datetime(df_indice['Date'])

# Transformar el DataFrame a un formato largo para análisis
df_melted = df_indice.melt(id_vars=['Date'], 
                            var_name='Estrategia', 
                            value_name='Rendimiento')

# Calcular métricas por estrategia
df_metrica = df_melted.groupby('Estrategia').agg(
    ARR=('Rendimiento', lambda x: ((1+np.mean(x)) ** (253)) - 1),
    Volatilidad=('Rendimiento', lambda x: np.std(x) * np.sqrt(253)),
    Sharpe=('Rendimiento', lambda x: ((((1+np.mean(x)) ** (253)) - 1)-0.05) / (np.std(x) * np.sqrt(253)))
).reset_index()

print("Métricas calculadas por estrategia:")
print(df_metrica)

# ANOVA para cada métrica
print("\nResultados ANOVA:")
for metric in ['ARR', 'Volatilidad', 'Sharpe']:
    groups = [df_melted[df_melted['Estrategia'] == strat]['Rendimiento'] for strat in df_melted['Estrategia'].unique()]
    f_stat, p_value = stats.f_oneway(*groups)
    print(f"{metric}: F={f_stat:.3f}, p-value={p_value:.3f}")

Métricas calculadas por estrategia:
           Estrategia       ARR  Volatilidad    Sharpe
0      Agente_BVC_Rnd  0.305993     0.294766  0.868461
1     Buy_Mantein_Rnd  1.150495     0.604075  1.821787
2     Media_Movil_Rnd -0.143060     0.605918 -0.318624
3  Revertion_Mean_Rnd -0.753035     0.433885 -1.850803

Resultados ANOVA:
ARR: F=3.365, p-value=0.018
Volatilidad: F=3.365, p-value=0.018
Sharpe: F=3.365, p-value=0.018


C:\Users\jumar\AppData\Local\Temp\ipykernel_952\1903470467.py:2: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



## Evolución Metricas

In [30]:
# Transformar el DataFrame a formato largo
df_melted = df_indice.melt(id_vars=['Date'], var_name='Estrategia', value_name='Rendimiento')

# Función para calcular métricas acumulativas
def calcular_metricas_acumulativas(df):
    df = df.sort_values('Date').copy()  # Asegurar orden cronológico
    df['ARR'] = (1 + df['Rendimiento']).cumprod() ** (253 / df.index.to_series().diff().fillna(1).cumsum()) - 1
    df['Volatilidad_acum'] = df['Rendimiento'].expanding().std() * np.sqrt(252)
    df['Sharpe_acum'] = (df['Rendimiento'].expanding().mean() / df['Rendimiento'].expanding().std()) * np.sqrt(252)
    return df

# Aplicar función por estrategia
df_time_metrics = df_melted.groupby('Estrategia', group_keys=False).apply(calcular_metricas_acumulativas)

fig = go.Figure()

for metric in ['ARR']:
    for strategy in df_time_metrics['Estrategia'].unique():
        df_temp = df_time_metrics[df_time_metrics['Estrategia'] == strategy]
        fig.add_trace(go.Scatter(
            x=df_temp['Date'],
            y=df_temp[metric],
            mode='lines',
            name=f"{strategy} - {metric}"
        ))

fig.update_layout(
    title={
        'text': f"<b>Evolución Acumulativa del ARR para {indice_referencia}</b>",
        'x':0.5,
        'xanchor': 'center'
    },
    yaxis_title="<b>ARR Acumulado</b>",
    legend=dict(
        title="<b>Estrategia</b>",
        orientation="h",
        yanchor="top",
        y=-0.15,
        xanchor="center",
        x=0.5
    ),
    template="ggplot2"
)


fig.show()

C:\Users\jumar\AppData\Local\Temp\ipykernel_952\1273305016.py:13: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



## ESTADISTICAS

In [31]:
from scipy.stats import shapiro

print("Prueba de Normalidad (Shapiro-Wilk):")
for estrategia in df_melted['Estrategia'].unique():
    data = df_melted[df_melted['Estrategia'] == estrategia]['Rendimiento']
    stat, p_value = shapiro(data)
    normalidad = "Normal" if p_value > 0.05 else "No normal"
    print(f"{estrategia}: Estadístico={stat:.3f}, p-value={p_value:.3f} ({normalidad})")


Prueba de Normalidad (Shapiro-Wilk):
Buy_Mantein_Rnd: Estadístico=0.644, p-value=0.000 (No normal)
Revertion_Mean_Rnd: Estadístico=0.238, p-value=0.000 (No normal)
Media_Movil_Rnd: Estadístico=0.644, p-value=0.000 (No normal)
Agente_BVC_Rnd: Estadístico=0.481, p-value=0.000 (No normal)


In [32]:
from scipy.stats import levene

# Preparar grupos para Levene
grupos = [df_melted[df_melted['Estrategia'] == estr]['Rendimiento'] 
          for estr in df_melted['Estrategia'].unique()]

# Ejecutar prueba de Levene
estadistico, p_value = levene(*grupos)

homocedasticidad = "Homocedástico" if p_value > 0.05 else "Heterocedástico"
print(f"\nPrueba de Homocedasticidad (Levene): Estadístico={estadistico:.3f}, p-value={p_value:.3f} ({homocedasticidad})")



Prueba de Homocedasticidad (Levene): Estadístico=11.051, p-value=0.000 (Heterocedástico)


In [ ]:
from scipy.stats import mannwhitneyu
from itertools import combinations
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# 'df_melted' con las columnas 'Estrategia' y 'Rendimiento'

# Extrae las estrategias únicas
estrategias = df_melted['Estrategia'].unique()

# Número de comparaciones (para la corrección de Bonferroni)
n_comparaciones = len(list(combinations(estrategias, 2)))

# Nivel de significancia ajustado (corrección de Bonferroni)
alpha_ajustado = 0.05 / n_comparaciones

# Realiza las comparaciones por pares con Mann-Whitney U
print("Resultados prueba Mann-Whitney U por pares (con corrección de Bonferroni):")
for estrategia1, estrategia2 in combinations(estrategias, 2):
    # Filtra los rendimientos por cada estrategia
    rend_1 = df_melted[df_melted['Estrategia'] == estrategia1]['Rendimiento']
    rend_2 = df_melted[df_melted['Estrategia'] == estrategia2]['Rendimiento']

    # Aplica Mann-Whitney U test
    stat, p_value = mannwhitneyu(rend_1, rend_2, alternative='two-sided')

    # Verifica la significancia estadística con la corrección de Bonferroni
    if p_value <= alpha_ajustado:
        significativo = "Sí"
    else:
        significativo = "No"

    print(f"{estrategia1} vs {estrategia2}: Estadístico U={stat:.3f}, p-valor={p_value:.3f}, Significativo={significativo}")

Resultados prueba Mann-Whitney U por pares (con corrección de Bonferroni):
Buy_Mantein_Rnd vs Revertion_Mean_Rnd: Estadístico U=34663.500, p-valor=0.033, Significativo=No
Buy_Mantein_Rnd vs Media_Movil_Rnd: Estadístico U=33373.500, p-valor=0.299, Significativo=No
Buy_Mantein_Rnd vs Agente_BVC_Rnd: Estadístico U=31810.500, p-valor=0.968, Significativo=No
Revertion_Mean_Rnd vs Media_Movil_Rnd: Estadístico U=30951.000, p-valor=0.557, Significativo=No
Revertion_Mean_Rnd vs Agente_BVC_Rnd: Estadístico U=28459.500, p-valor=0.002, Significativo=Sí
Media_Movil_Rnd vs Agente_BVC_Rnd: Estadístico U=29785.500, p-valor=0.172, Significativo=No


# BANCOLOMBIA

## DATOS

In [ ]:
indice_referencia = "PFBCOLOM.CL"
df_indice = pd.read_csv(f"Resultados_Experimento/{indice_referencia}/stock_sugerencias.csv")
df_indice=df_indice.loc[: ,["Date","Buy_Mantein_Rnd","Revertion_Mean_Rnd","Media_Movil_Rnd","Agente_BVC_Rnd"]]
df_indice.head(5)

,Date,Buy_Mantein_Rnd,Revertion_Mean_Rnd,Media_Movil_Rnd,Agente_BVC_Rnd
0,2024-01-01 00:00:00-05:00,0.000000,0.000000,0.000000,0.000000
1,2024-01-02 00:00:00-05:00,0.019385,0.000000,0.019385,0.000000
2,2024-01-03 00:00:00-05:00,-0.006557,0.000000,-0.006557,0.000000
3,2024-01-04 00:00:00-05:00,0.064686,-0.064686,0.064686,0.064686
4,2024-01-05 00:00:00-05:00,0.029138,-0.029138,0.029138,0.029138


## Metricas y ANOVA

In [ ]:
# df_indice ya está cargado con las columnas mencionadas
df_indice['Date'] = pd.to_datetime(df_indice['Date'])

# Transformar el DataFrame a un formato largo para análisis
df_melted = df_indice.melt(id_vars=['Date'], 
                            var_name='Estrategia', 
                            value_name='Rendimiento')

# Calcular métricas por estrategia
df_metrica = df_melted.groupby('Estrategia').agg(
    ARR=('Rendimiento', lambda x: ((1+np.mean(x)) ** (253)) - 1),
    Volatilidad=('Rendimiento', lambda x: np.std(x) * np.sqrt(253)),
    Sharpe=('Rendimiento', lambda x: ((((1+np.mean(x)) ** (253)) - 1)-0.05) / (np.std(x) * np.sqrt(253)))
).reset_index()

print("Métricas calculadas por estrategia:")
print(df_metrica)

# ANOVA para cada métrica
print("\nResultados ANOVA:")
for metric in ['ARR', 'Volatilidad', 'Sharpe']:
    groups = [df_melted[df_melted['Estrategia'] == strat]['Rendimiento'] for strat in df_melted['Estrategia'].unique()]
    f_stat, p_value = stats.f_oneway(*groups)
    print(f"{metric}: F={f_stat:.3f}, p-value={p_value:.3f}")

Métricas calculadas por estrategia:
           Estrategia       ARR  Volatilidad    Sharpe
0      Agente_BVC_Rnd  0.085744     0.128263  0.278681
1     Buy_Mantein_Rnd  0.341164     0.232590  1.251831
2     Media_Movil_Rnd  0.075982     0.233277  0.111380
3  Revertion_Mean_Rnd -0.358852     0.119566 -3.419459

Resultados ANOVA:
ARR: F=2.819, p-value=0.038
Volatilidad: F=2.819, p-value=0.038
Sharpe: F=2.819, p-value=0.038


C:\Users\jumar\AppData\Local\Temp\ipykernel_952\1903470467.py:2: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



## Evolución Metricas

In [22]:
# Transformar el DataFrame a formato largo
df_melted = df_indice.melt(id_vars=['Date'], var_name='Estrategia', value_name='Rendimiento')

# Función para calcular métricas acumulativas
def calcular_metricas_acumulativas(df):
    df = df.sort_values('Date').copy()  # Asegurar orden cronológico
    df['ARR'] = (1 + df['Rendimiento']).cumprod() ** (253 / df.index.to_series().diff().fillna(1).cumsum()) - 1
    df['Volatilidad_acum'] = df['Rendimiento'].expanding().std() * np.sqrt(252)
    df['Sharpe_acum'] = (df['Rendimiento'].expanding().mean() / df['Rendimiento'].expanding().std()) * np.sqrt(252)
    return df

# Aplicar función por estrategia
df_time_metrics = df_melted.groupby('Estrategia', group_keys=False).apply(calcular_metricas_acumulativas)

fig = go.Figure()

for metric in ['ARR']:
    for strategy in df_time_metrics['Estrategia'].unique():
        df_temp = df_time_metrics[df_time_metrics['Estrategia'] == strategy]
        fig.add_trace(go.Scatter(
            x=df_temp['Date'],
            y=df_temp[metric],
            mode='lines',
            name=f"{strategy} - {metric}"
        ))

fig.update_layout(
    title={
        'text': f"<b>Evolución Acumulativa del ARR para {indice_referencia}</b>",
        'x':0.5,
        'xanchor': 'center'
    },
    yaxis_title="<b>ARR Acumulado</b>",
    legend=dict(
        title="<b>Estrategia</b>",
        orientation="h",
        yanchor="top",
        y=-0.15,
        xanchor="center",
        x=0.5
    ),
    template="ggplot2"
)


fig.show()

C:\Users\jumar\AppData\Local\Temp\ipykernel_952\1273305016.py:13: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



## ESTADISTICAS

In [23]:
from scipy.stats import shapiro

print("Prueba de Normalidad (Shapiro-Wilk):")
for estrategia in df_melted['Estrategia'].unique():
    data = df_melted[df_melted['Estrategia'] == estrategia]['Rendimiento']
    stat, p_value = shapiro(data)
    normalidad = "Normal" if p_value > 0.05 else "No normal"
    print(f"{estrategia}: Estadístico={stat:.3f}, p-value={p_value:.3f} ({normalidad})")


Prueba de Normalidad (Shapiro-Wilk):
Buy_Mantein_Rnd: Estadístico=0.963, p-value=0.000 (No normal)
Revertion_Mean_Rnd: Estadístico=0.252, p-value=0.000 (No normal)
Media_Movil_Rnd: Estadístico=0.963, p-value=0.000 (No normal)
Agente_BVC_Rnd: Estadístico=0.610, p-value=0.000 (No normal)


In [24]:
from scipy.stats import levene

# Preparar grupos para Levene
grupos = [df_melted[df_melted['Estrategia'] == estr]['Rendimiento'] 
          for estr in df_melted['Estrategia'].unique()]

# Ejecutar prueba de Levene
estadistico, p_value = levene(*grupos)

homocedasticidad = "Homocedástico" if p_value > 0.05 else "Heterocedástico"
print(f"\nPrueba de Homocedasticidad (Levene): Estadístico={estadistico:.3f}, p-value={p_value:.3f} ({homocedasticidad})")



Prueba de Homocedasticidad (Levene): Estadístico=69.109, p-value=0.000 (Heterocedástico)


In [ ]:
from scipy.stats import mannwhitneyu
from itertools import combinations
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# 'df_melted' con las columnas 'Estrategia' y 'Rendimiento'

# Extrae las estrategias únicas
estrategias = df_melted['Estrategia'].unique()

# Número de comparaciones (para la corrección de Bonferroni)
n_comparaciones = len(list(combinations(estrategias, 2)))

# Nivel de significancia ajustado (corrección de Bonferroni)
alpha_ajustado = 0.05 / n_comparaciones

# Realiza las comparaciones por pares con Mann-Whitney U
print("Resultados prueba Mann-Whitney U por pares (con corrección de Bonferroni):")
for estrategia1, estrategia2 in combinations(estrategias, 2):
    # Filtra los rendimientos por cada estrategia
    rend_1 = df_melted[df_melted['Estrategia'] == estrategia1]['Rendimiento']
    rend_2 = df_melted[df_melted['Estrategia'] == estrategia2]['Rendimiento']

    # Aplica Mann-Whitney U test
    stat, p_value = mannwhitneyu(rend_1, rend_2, alternative='two-sided')

    # Verifica la significancia estadística con la corrección de Bonferroni
    if p_value <= alpha_ajustado:
        significativo = "Sí"
    else:
        significativo = "No"

    print(f"{estrategia1} vs {estrategia2}: Estadístico U={stat:.3f}, p-valor={p_value:.3f}, Significativo={significativo}")

Resultados prueba Mann-Whitney U por pares (con corrección de Bonferroni):
Buy_Mantein_Rnd vs Revertion_Mean_Rnd: Estadístico U=34560.000, p-valor=0.096, Significativo=No
Buy_Mantein_Rnd vs Media_Movil_Rnd: Estadístico U=33069.500, p-valor=0.517, Significativo=No
Buy_Mantein_Rnd vs Agente_BVC_Rnd: Estadístico U=32738.000, p-valor=0.645, Significativo=No
Revertion_Mean_Rnd vs Media_Movil_Rnd: Estadístico U=30898.000, p-valor=0.472, Significativo=No
Revertion_Mean_Rnd vs Agente_BVC_Rnd: Estadístico U=29801.000, p-valor=0.047, Significativo=No
Media_Movil_Rnd vs Agente_BVC_Rnd: Estadístico U=31348.500, p-valor=0.680, Significativo=No


# ARGOS

## DATOS

In [13]:
indice_referencia = "CEMARGOS.CL"
df_indice = pd.read_csv(f"Resultados_Experimento/{indice_referencia}/stock_sugerencias.csv")
df_indice=df_indice.loc[: ,["Date","Buy_Mantein_Rnd","Revertion_Mean_Rnd","Media_Movil_Rnd","Agente_BVC_Rnd"]]
df_indice.head(5)

,Date,Buy_Mantein_Rnd,Revertion_Mean_Rnd,Media_Movil_Rnd,Agente_BVC_Rnd
0,2024-01-01 00:00:00-05:00,0.000000,0.000000,0.000000,0.000000
1,2024-01-02 00:00:00-05:00,0.008183,0.000000,0.008183,0.000000
2,2024-01-03 00:00:00-05:00,-0.001623,0.000000,-0.001623,0.000000
3,2024-01-04 00:00:00-05:00,0.014634,0.000000,0.014634,0.014634
4,2024-01-05 00:00:00-05:00,0.030449,-0.030449,0.030449,0.030449


## Metricas

In [ ]:
# df_indice ya está cargado con las columnas mencionadas
df_indice['Date'] = pd.to_datetime(df_indice['Date'])

# Transformar el DataFrame a un formato largo para análisis
df_melted = df_indice.melt(id_vars=['Date'], 
                            var_name='Estrategia', 
                            value_name='Rendimiento')

# Calcular métricas por estrategia
df_metrica = df_melted.groupby('Estrategia').agg(
    ARR=('Rendimiento', lambda x: ((1+np.mean(x)) ** (253)) - 1),
    Volatilidad=('Rendimiento', lambda x: np.std(x) * np.sqrt(253)),
    Sharpe=('Rendimiento', lambda x: ((((1+np.mean(x)) ** (253)) - 1)-0.05) / (np.std(x) * np.sqrt(253)))
).reset_index()

print("Métricas calculadas por estrategia:")
print(df_metrica)

Métricas calculadas por estrategia:
           Estrategia       ARR  Volatilidad    Sharpe
0      Agente_BVC_Rnd  0.134101     0.175430  0.479401
1     Buy_Mantein_Rnd  0.838392     0.305176  2.583399
2     Media_Movil_Rnd  0.264525     0.307219  0.698280
3  Revertion_Mean_Rnd -0.571957     0.189732 -3.278084


C:\Users\jumar\AppData\Local\Temp\ipykernel_952\2130635267.py:2: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



## Evolución Metricas

In [15]:
# Transformar el DataFrame a formato largo
df_melted = df_indice.melt(id_vars=['Date'], var_name='Estrategia', value_name='Rendimiento')

# Función para calcular métricas acumulativas
def calcular_metricas_acumulativas(df):
    df = df.sort_values('Date').copy()  # Asegurar orden cronológico
    df['ARR'] = (1 + df['Rendimiento']).cumprod() ** (253 / df.index.to_series().diff().fillna(1).cumsum()) - 1
    df['Volatilidad_acum'] = df['Rendimiento'].expanding().std() * np.sqrt(252)
    df['Sharpe_acum'] = (df['Rendimiento'].expanding().mean() / df['Rendimiento'].expanding().std()) * np.sqrt(252)
    return df

# Aplicar función por estrategia
df_time_metrics = df_melted.groupby('Estrategia', group_keys=False).apply(calcular_metricas_acumulativas)

fig = go.Figure()

for metric in ['ARR']:
    for strategy in df_time_metrics['Estrategia'].unique():
        df_temp = df_time_metrics[df_time_metrics['Estrategia'] == strategy]
        fig.add_trace(go.Scatter(
            x=df_temp['Date'],
            y=df_temp[metric],
            mode='lines',
            name=f"{strategy} - {metric}"
        ))

fig.update_layout(
    title={
        'text': f"<b>Evolución Acumulativa del ARR para {indice_referencia}</b>",
        'x':0.5,
        'xanchor': 'center'
    },
    yaxis_title="<b>ARR Acumulado</b>",
    legend=dict(
        title="<b>Estrategia</b>",
        orientation="h",
        yanchor="top",
        y=-0.15,
        xanchor="center",
        x=0.5
    ),
    template="ggplot2"
)


fig.show()

C:\Users\jumar\AppData\Local\Temp\ipykernel_952\1273305016.py:13: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



## ESTADISTICAS

In [16]:
from scipy.stats import shapiro

print("Prueba de Normalidad (Shapiro-Wilk):")
for estrategia in df_melted['Estrategia'].unique():
    data = df_melted[df_melted['Estrategia'] == estrategia]['Rendimiento']
    stat, p_value = shapiro(data)
    normalidad = "Normal" if p_value > 0.05 else "No normal"
    print(f"{estrategia}: Estadístico={stat:.3f}, p-value={p_value:.3f} ({normalidad})")


Prueba de Normalidad (Shapiro-Wilk):
Buy_Mantein_Rnd: Estadístico=0.932, p-value=0.000 (No normal)
Revertion_Mean_Rnd: Estadístico=0.369, p-value=0.000 (No normal)
Media_Movil_Rnd: Estadístico=0.933, p-value=0.000 (No normal)
Agente_BVC_Rnd: Estadístico=0.580, p-value=0.000 (No normal)


In [17]:
from scipy.stats import levene

# Preparar grupos para Levene
grupos = [df_melted[df_melted['Estrategia'] == estr]['Rendimiento'] 
          for estr in df_melted['Estrategia'].unique()]

# Ejecutar prueba de Levene
estadistico, p_value = levene(*grupos)

homocedasticidad = "Homocedástico" if p_value > 0.05 else "Heterocedástico"
print(f"\nPrueba de Homocedasticidad (Levene): Estadístico={estadistico:.3f}, p-value={p_value:.3f} ({homocedasticidad})")



Prueba de Homocedasticidad (Levene): Estadístico=41.758, p-value=0.000 (Heterocedástico)


In [ ]:
from scipy.stats import mannwhitneyu
from itertools import combinations
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# 'df_melted' con las columnas 'Estrategia' y 'Rendimiento'

# Extrae las estrategias únicas
estrategias = df_melted['Estrategia'].unique()

# Número de comparaciones (para la corrección de Bonferroni)
n_comparaciones = len(list(combinations(estrategias, 2)))

# Nivel de significancia ajustado (corrección de Bonferroni)
alpha_ajustado = 0.05 / n_comparaciones

# Realiza las comparaciones por pares con Mann-Whitney U
print("Resultados prueba Mann-Whitney U por pares (con corrección de Bonferroni):")
for estrategia1, estrategia2 in combinations(estrategias, 2):
    # Filtra los rendimientos por cada estrategia
    rend_1 = df_melted[df_melted['Estrategia'] == estrategia1]['Rendimiento']
    rend_2 = df_melted[df_melted['Estrategia'] == estrategia2]['Rendimiento']

    # Aplica Mann-Whitney U test
    stat, p_value = mannwhitneyu(rend_1, rend_2, alternative='two-sided')

    # Verifica la significancia estadística con la corrección de Bonferroni
    if p_value <= alpha_ajustado:
        significativo = "Sí"
    else:
        significativo = "No"

    print(f"{estrategia1} vs {estrategia2}: Estadístico U={stat:.3f}, p-valor={p_value:.3f}, Significativo={significativo}")

Resultados prueba Mann-Whitney U por pares (con corrección de Bonferroni):
Buy_Mantein_Rnd vs Revertion_Mean_Rnd: Estadístico U=37187.000, p-valor=0.001, Significativo=Sí
Buy_Mantein_Rnd vs Media_Movil_Rnd: Estadístico U=33391.000, p-valor=0.399, Significativo=No
Buy_Mantein_Rnd vs Agente_BVC_Rnd: Estadístico U=33764.000, p-valor=0.270, Significativo=No
Revertion_Mean_Rnd vs Media_Movil_Rnd: Estadístico U=28263.000, p-valor=0.016, Significativo=No
Revertion_Mean_Rnd vs Agente_BVC_Rnd: Estadístico U=27988.500, p-valor=0.001, Significativo=Sí
Media_Movil_Rnd vs Agente_BVC_Rnd: Estadístico U=32258.500, p-valor=0.874, Significativo=No


# Grafico Documento

In [9]:
fig = go.Figure()

for metric in ['ARR']:
    for strategy in df_time_metrics['Estrategia'].unique():
        df_temp = df_time_metrics[df_time_metrics['Estrategia'] == strategy]
        strategy= "Reversión_Media_Rnd" if strategy == "Revertion_Mean_Rnd" else "Comprar_Mantener_Rnd" if strategy == "Buy_Mantein_Rnd" else strategy 
        fig.add_trace(go.Scatter(
            x=df_temp['Date'],
            y=df_temp[metric],
            mode='lines',
            name=f"{strategy}",
            line=dict(width=4)
        ))

fig.update_layout(
    title={
        'text': f"<b>Evolución Acumulativa del ARR para {indice_referencia}</b>",
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 36}
    },
    xaxis=dict(
        tickfont=dict(size=30),
        range=["2024-01-20", df_time_metrics['Date'].max()]
    ),
    yaxis=dict(
        title=dict(text="<b>ARR Acumulado</b>", font=dict(size=30)),
        tickfont=dict(size=30),
        range=[-1, 3.5]
    ),
    legend=dict(
        title="<b>Estrategia</b>",
        orientation="h",
        yanchor="top",
        y=-0.2,
        xanchor="center",
        x=0.5,
        font=dict(size=30, color='black')
    ),
    margin=dict(l=60, r=60, t=100, b=100),
    template="ggplot2"
)

fig.write_image("newplot.png", width=1920, height=1080, scale=3)

